In [1]:
# Importamos bibliotecas
import requests
from lxml import html
import time
import random
import pandas as pd

In [2]:
#macros, se podren pasar como argumentos en futuras versiones
#hasta = '2021-12-31'
#desde = '2021-01-01'
area = 'advantage actor critic'
fuente = 'arxiv'
# numero de paginas a revisar (paginas de 10 elementos)
num_de_paginas = 4 

In [3]:
# Url original 
#https://scholar.google.com/scholar?q=advantage+actor+critic+source%3Aarxiv&oq=

In [4]:
pagina = 'https://scholar.google.com/scholar?q='+area.replace(" ", "+")+'+source%3A'+fuente+'&oq='
headers = requests.utils.default_headers()

headers.update(
    {
        'User-Agent': 'Chrome/96.0.4664.93',
    }
)

req =  requests.get(pagina, headers=headers)
if req.status_code == 429:
    print('wait' +req.headers.get('Retry-After'))
    time.sleep(int(req.headers.get('Retry-After'))+1)
    req =  requests.get(pagina, headers=headers)
print(req)

#Convertimos a tree 
arbol_html = html.fromstring(req.content)

<Response [200]>


In [5]:
# start=10 


papers = []


for i in range (num_de_paginas):
    print('Empiezo vuelta '+str(i)+' :'+time.strftime("%H:%M:%S", time.gmtime()))
    req =  requests.get(pagina, headers=headers)
    if req.status_code == 429:
        print('wait' +req.headers.get('Retry-After'))
        time.sleep(int(req.headers.get('Retry-After'))+1)
        req =  requests.get(pagina, headers=headers)
    #Convertimos a tree 
    arbol_html = html.fromstring(req.content)
    temp_res = arbol_html.xpath('//h3 [@class ="gs_rt"]/a/@href')
    for i in range(len(temp_res)):
        pag = temp_res[i]
        time.sleep(random.randint(1,2)*0.5)
        paper_res =  requests.get(pag, headers=headers)
        paper_dic = {}
        if paper_res.status_code == 429:
            print('wait' +paper_res.headers.get('Retry-After'))
            time.sleep(int(jugador_req.headers.get('Retry-After'))+1)
            paper_res =  requests.get(pag, headers=headers)
        paper_arbol_html = html.fromstring(paper_res.content)
        # especifico de la pagina
        paper_dic["fuente"] = "arxiv.org"
        paper_dic["titulo"] = paper_arbol_html.xpath('//h1 [@class ="title mathjax"]/text()')
        paper_dic["autores"] = paper_arbol_html.xpath('//div [@class ="authors"]/a/text()')
        paper_dic["abstract"] = paper_arbol_html.xpath('//blockquote [@class ="abstract mathjax"]/text()')
        paper_dic["clase_primaria"] = paper_arbol_html.xpath('//span [@class ="primary-subject"]/text()')
        paper_dic["otras_clases"] = paper_arbol_html.xpath('//td [@class ="tablecell subjects"]/text()')
        paper_dic["clase_primaria"] = paper_arbol_html.xpath('//span [@class ="primary-subject"]/text()')
        #generico
        paper_dic["n_citaciones"] = arbol_html.xpath('//a [@href ="'+pag+'"]/parent::*/parent::*/div [@class = "gs_fl"]/a [contains(text(),"Citado")]/text()')
        paper_dic["citado"] = arbol_html.xpath('//a [@href ="'+pag+'"]/parent::*/parent::*/div [@class = "gs_fl"]/a [contains(text(),"Citado")]/@href')
        papers.append(paper_dic)
        pagina = 'https://scholar.google.com/scholar?start='+str(10*(i+1))+'&q='+area.replace(" ", "+")+'+source%3A'+fuente+'&oq='

Empiezo vuelta 0 :22:29:39
Empiezo vuelta 1 :22:29:54
Empiezo vuelta 2 :22:30:04
Empiezo vuelta 3 :22:30:18


In [6]:
titulos = []
autores = []
abstractos = []
clase_pri = []
clase_otr = []
for papel in papers:
    try:
        titulos.append(papel["titulo"][0])
        autores.append(papel["autores"][0])
        abstractos.append(papel["abstract"][1])
        clase_pri.append(papel["clase_primaria"][0])
    except:
        continue
    try:
        clase_otr.append(papel["otras_clases"][1])
    except:
        clase_otr.append("")

In [7]:
len(autores)

34

In [8]:
len(titulos)

34

In [9]:
data = {'titulo': titulos,
        'autores': autores,
        'abstract': abstractos,
        'clase_pri': clase_pri,
        'clase_otr': clase_otr
        }

df = pd.DataFrame(data)

In [10]:
filepath = 'data_'+area+".csv"
df.to_csv(filepath,index=False)  

In [11]:
df.dtypes

titulo       object
autores      object
abstract     object
clase_pri    object
clase_otr    object
dtype: object

In [12]:
df.head()

,titulo,autores,abstract,clase_pri,clase_otr
0,Reinforcement Learning through Asynchronous Ad...,Mohammad Babaeizadeh,We introduce a hybrid CPU/GPU version of the...,Machine Learning (cs.LG),
1,Altruistic Maneuver Planning for Cooperative A...,Behrad Toghi,With the adoption of autonomous vehicles on ...,Robotics (cs.RO),
2,Towards Understanding Asynchronous Advantage A...,Han Shen,Asynchronous and parallel implementation of ...,Machine Learning (cs.LG),; Optimization and Control (math.OC)
3,The Advantage Regret-Matching Actor-Critic,Audrūnas Gruslys,Regret minimization has played a key role in...,Artificial Intelligence (cs.AI),; Machine Learning (cs.LG)
4,Actor-Critic Sequence Training for Image Capti...,Li Zhang,Generating natural language descriptions of ...,Computer Vision and Pattern Recognition (cs.CV),


In [16]:
df['id'] = df.index

In [18]:
df.head()

,titulo,autores,abstract,clase_pri,clase_otr,id
0,Reinforcement Learning through Asynchronous Ad...,Mohammad Babaeizadeh,We introduce a hybrid CPU/GPU version of the...,Machine Learning (cs.LG),,0
1,Altruistic Maneuver Planning for Cooperative A...,Behrad Toghi,With the adoption of autonomous vehicles on ...,Robotics (cs.RO),,1
2,Towards Understanding Asynchronous Advantage A...,Han Shen,Asynchronous and parallel implementation of ...,Machine Learning (cs.LG),; Optimization and Control (math.OC),2
3,The Advantage Regret-Matching Actor-Critic,Audrūnas Gruslys,Regret minimization has played a key role in...,Artificial Intelligence (cs.AI),; Machine Learning (cs.LG),3
4,Actor-Critic Sequence Training for Image Capti...,Li Zhang,Generating natural language descriptions of ...,Computer Vision and Pattern Recognition (cs.CV),,4


### Conectar a Neo4j Desktop

In [13]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
df = pd.read_csv('../AplicacionesYTendencias/ProyectoAyT/UEM-Analytics-G22-3/data_advantage actor critic.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: '../AplicacionesYTendencias/ProyectoAyT/UEM-Analytics-G22-3/data_advantage actor critic.csv'

In [14]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [15]:
conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="usermate",              
                       pwd="glass")

### Creación BBDD

In [25]:
conn.query('CREATE CONSTRAINT papers IF NOT EXISTS FOR (p:Paper) REQUIRE p.id IS UNIQUE')
conn.query('CREATE CONSTRAINT authors IF NOT EXISTS FOR (a:Author) REQUIRE a.name IS UNIQUE')
conn.query('CREATE CONSTRAINT categories IF NOT EXISTS FOR (c:Category) REQUIRE c.category IS UNIQUE')

[]

In [26]:
def add_categories(categories):
    # Adds category nodes to the Neo4j graph.
    query = '''
            UNWIND $rows AS row
            MERGE (c:Category {category: row.category})
            RETURN count(*) as total
            '''
    return conn.query(query, parameters = {'rows':categories.to_dict('records')})


def add_authors(rows, batch_size=10000):
    # Adds author nodes to the Neo4j graph as a batch job.
    query = '''
            UNWIND $rows AS row
            MERGE (:Author {name: row.author})
            RETURN count(*) as total
            '''
    return insert_data(query, rows, batch_size)


def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.
    
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):

        res = conn.query(query, 
                         parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result

In [27]:
def add_papers(rows, batch_size=5000):
   # Adds paper nodes and (:Author)--(:Paper) and 
   # (:Paper)--(:Category) relationships to the Neo4j graph as a 
   # batch job.
 
   query = '''
   UNWIND $rows as row
   MERGE (p:Paper {id:row.id}) ON CREATE SET p.title = row.title
 
   // connect categories
   WITH row, p
   UNWIND row.category_list AS category_name
   MATCH (c:Category {category: category_name})
   MERGE (p)-[:IN_CATEGORY]->(c)
 
   // connect authors
   WITH distinct row, p // reduce cardinality
   UNWIND row.cleaned_authors_list AS author
   MATCH (a:Author {name: author})
   MERGE (a)-[:AUTHORED]->(p)
   RETURN count(distinct p) as total
   '''
 
   return insert_data(query, rows, batch_size)

In [29]:
df.head()

,titulo,autores,abstract,clase_pri,clase_otr,id
0,Reinforcement Learning through Asynchronous Ad...,Mohammad Babaeizadeh,We introduce a hybrid CPU/GPU version of the...,Machine Learning (cs.LG),,0
1,Altruistic Maneuver Planning for Cooperative A...,Behrad Toghi,With the adoption of autonomous vehicles on ...,Robotics (cs.RO),,1
2,Towards Understanding Asynchronous Advantage A...,Han Shen,Asynchronous and parallel implementation of ...,Machine Learning (cs.LG),; Optimization and Control (math.OC),2
3,The Advantage Regret-Matching Actor-Critic,Audrūnas Gruslys,Regret minimization has played a key role in...,Artificial Intelligence (cs.AI),; Machine Learning (cs.LG),3
4,Actor-Critic Sequence Training for Image Capti...,Li Zhang,Generating natural language descriptions of ...,Computer Vision and Pattern Recognition (cs.CV),,4


In [31]:
categories = pd.DataFrame(df[['clase_pri']])
categories.rename(columns={'clase_pri':'category'},
                  inplace=True)
categories = categories.explode('category') \
                       .drop_duplicates(subset=['category'])

authors = pd.DataFrame(df[['autores']])
authors.rename(columns={'autores':'author'},
               inplace=True)
authors=authors.explode('author').drop_duplicates(subset=['author'])
add_categories(categories)
add_authors(authors)
add_papers(df)